In [44]:
import pandas as pd
import pandasql as psql
import numpy as np
import json
from shapely.geometry import Point
from shapely.geometry import Polygon
from scipy.spatial import cKDTree

In [3]:
neigh_bounds = '../SimplifiedGeoJson/nbounds_by_boro.json'
pct_bounds = '../SimplifiedGeoJson/precinct_bounds_by_borough.json'

In [4]:
with open(neigh_bounds, 'r') as fp:
    nbounds = json.load(fp)

In [5]:
with open(pct_bounds, 'r') as fp:
    pbounds = json.load(fp)

### Get neighbourhood name for listings currently listed as "New Dev"

In [8]:
def area_form_coord(long, lat, boro):
    pt = Point(long, lat)

    for neigh in nbounds[boro]:
        ncoords = neigh['coords'][0][0]
        coords = [tuple(x) for x in ncoords]
        poly = Polygon(coords)

        if poly.contains(pt):
            return neigh['name']
    
    return None

In [ ]:
#     daf.at[idx, 'Address'] = data[-1]
#     daf.at[idx, 'ZipCode'] = data[2]
#     daf.at[idx, 'Lat'] = data[1]
#     daf.at[idx, 'Long'] = data[0]

# daf.to_csv(df_dest, index=False)
# return

#### Brooklyn

In [9]:
bk_df = pd.DataFrame(pd.read_csv('bklyn_df.csv'))

In [11]:
not_found = 0
for idx, row in bk_df.iterrows():
    area, lon, lat = row['Area'], row['Long'], row['Lat']
    if area.lower() == 'new_dev':
        afb = area_form_coord(lon, lat, 'brooklyn')

        if afb is None:
            not_found+=1
        
        bk_df.at[idx, 'Area'] = afb

not_found

25

In [15]:
bk_df.to_csv('bklyn_df.csv', index=False)

#### Bronx

In [22]:
bx_df = pd.DataFrame(pd.read_csv('bronx_df.csv'))

In [21]:
bx_df[bx_df['Area'] == 'New_Dev'].count()

Borough      121
Area         121
Address      121
Num_Beds     121
Num_Baths    121
Sq-Ft        121
Rent         121
Long         121
Lat          121
ZipCode      121
dtype: int64

In [23]:
not_found = 0
for idx, row in bx_df.iterrows():
    area, lon, lat = row['Area'], row['Long'], row['Lat']
    if area.lower() == 'new_dev':
        afb = area_form_coord(lon, lat, 'bronx')

        if afb is None:
            not_found+=1
        
        bx_df.at[idx, 'Area'] = afb

not_found

117

In [25]:
bx_df.to_csv('bronx_df.csv', index=False)

#### Manhattan

In [26]:
ma_df = pd.DataFrame(pd.read_csv('manhatt_df.csv'))

In [27]:
ma_df[ma_df['Area'] == 'New_Dev'].count()

Borough      741
Area         741
Address      741
Num_Beds     741
Num_Baths    741
Sq-Ft        741
Rent         741
Long         729
Lat          729
ZipCode      729
dtype: int64

In [28]:
not_found = 0
for idx, row in ma_df.iterrows():
    area, lon, lat = row['Area'], row['Long'], row['Lat']
    if area.lower() == 'new_dev':
        afb = area_form_coord(lon, lat, 'manhattan')

        if afb is None:
            not_found+=1
        
        ma_df.at[idx, 'Area'] = afb

not_found

356

In [29]:
ma_df.to_csv('manhatt_df.csv', index=False)

#### Queens

In [30]:
qu_df = pd.DataFrame(pd.read_csv('queens_df.csv'))

In [31]:
qu_df[qu_df['Area'] == 'New_Dev'].count()

Borough      262
Area         262
Address      262
Num_Beds     262
Num_Baths    262
Sq-Ft        262
Rent         262
Long         261
Lat          261
ZipCode      261
dtype: int64

In [32]:
not_found = 0
for idx, row in qu_df.iterrows():
    area, lon, lat = row['Area'], row['Long'], row['Lat']
    if area.lower() == 'new_dev':
        afb = area_form_coord(lon, lat, 'queens')

        if afb is None:
            not_found+=1
        
        qu_df.at[idx, 'Area'] = afb

not_found

1

In [33]:
qu_df.to_csv('queens_df.csv', index=False)

#### Staten-Island

In [34]:
st_df = pd.DataFrame(pd.read_csv('stat_df.csv'))

In [35]:
st_df[st_df['Area'] == 'New_Dev'].count()

Borough      0
Area         0
Address      0
Num_Beds     0
Num_Baths    0
Sq-Ft        0
Rent         0
Long         0
Lat          0
ZipCode      0
dtype: int64

### Add Precinct Data

In [61]:
boro_dfs = ['stat_df.csv', 'queens_df.csv', 'manhatt_df.csv', 'bronx_df.csv', 'bklyn_df.csv']
# boro_dfs = ['stat_df.csv']
boros = ['staten-island', 'queens', 'manhattan', 'bronx', 'brooklyn']


In [41]:
def pct_from_bounds(long, lat, boro):
    pt = Point(long, lat)

    pcts = pbounds[boro].keys()

    for pct in pcts:
        pcoords = pbounds[boro][pct]['geometry']['coordinates'][0][0]
        coords = [tuple(x) for x in pcoords]
        poly = Polygon(coords)

        if poly.contains(pt):
            return pct
        elif poly.touches(pt):
            return pct
    
    return None

In [42]:
for bdf in boro_dfs:
    cur_df = pd.DataFrame(pd.read_csv(bdf))
    cur_df['Precinct'] = None

    for idx, row in cur_df.iterrows():
        borough, lon, lat = row['Borough'], row['Long'], row['Lat']
        if lon is not None and lat is not None:
            pct_num = pct_from_bounds(lon, lat, borough)

            cur_df.at[idx, 'Precinct'] = pct_num
    
    cur_df.to_csv(bdf, index=False)

    

#### For instances missing precinct and neighbourhood name, use grid search to approximate.

In [48]:
# first create search trees
bmap_ckds = {}

In [49]:
def create_ckdtrees(bmap, boro):
    if boro not in bmap:
        bmap[boro] = {}

    for pct in pbounds[boro]:
        bmap[boro][pct] = cKDTree(pbounds[boro][pct]['geometry']['coordinates'][0][0])
    
    return

In [50]:
for b in boros:
    create_ckdtrees(bmap_ckds, b)

In [81]:
def pct_grid_search(boro, lon, lat):
    nearests = []

    for pct in bmap_ckds[boro]:
        dist, _ = bmap_ckds[boro][pct].query([lon, lat], k=1)
        nearests.append([dist, pct])
    
    nearests.sort(key = lambda x: x[0])

    return nearests[0][1]

    

In [82]:
for bdf in boro_dfs:
    cur_df = pd.DataFrame(pd.read_csv(bdf))

    # cur_df['Precinct'].replace(r'^\s*$', np.nan, regex=True, inplace=True)
    print(cur_df['Precinct'].isna().sum())

    for idx, row in cur_df.iterrows():
        borough, lon, lat, precinct = row['Borough'], row['Long'], row['Lat'], row['Precinct']
        if not np.isnan(lon) and not np.isnan(lat) and np.isnan(precinct):
            pct_num = pct_grid_search(borough, lon, lat)
            cur_df.at[idx, 'Precinct'] = pct_num
    
    cur_df.to_csv(bdf, index=False)

22
1245
2874
288
521
